# Generating Autoprotocol json output from ESCALATE experiments

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import escalateclient
import importlib

In [24]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'nsmina'
password = 'password11'
client = escalateclient.ESCALATEClient(server_url, username, password)

<Response [200]>


In [25]:
#dependencies and helper functions

import json

import pint
from pint import UnitRegistry
units = UnitRegistry()
Q_ = units.Quantity

def format_units(quantity_val, quantity_unit):
    '''[summary] Standardizes units in Autoprotocol format
    Args
        quantity_val([float]): magnitude of a quantity
        quantity_unit([str]): unit associated with the quantity - can be full word or abbreviation
    
    Returns
        [str]: quantity and unit formatted for Autoprotocol (e.g "50:microliter")
    
    '''
    
    unit= str(Q_(quantity_val, quantity_unit).units)
    if unit=='degC':
        unit="celcius"
    return str(quantity_val)+':'+unit

def export_json(json_output, filename):
    '''[summary] Saves dictionary to a json file
    Args
        json_output([dic]): contains the desired json output
        filename([str]): desired name under which to save the file
    
    Returns
        [str]: quantity and unit formatted for Autoprotocol (e.g "50:microliter")
    
    '''
    data={'instructions': json_output}

    json_object = json.dumps(data, indent = 4)

    with open("{}.json".format(filename), "w") as outfile:
        outfile.write(json_object)
        
def get_uuid(url):
    '''[summary] Parses a URL to obtain UUID
    Args
        url ([str]): from ESCALATE database
    
    Returns
        [str]: UUID of object associated with the url
    
    '''
    uuid= url.split('/')[5]
    return uuid

In [43]:
def escalate_to_autoprotocol(exp_name):
    
    #obtain the action sequences for the experiment instance
    action_sequences=client.get_or_create(endpoint='experiment-instance', 
                     data={'description': exp_name})[0]['action_sequence']
    
    #obtain actions and action units
    actions=[]
    for ac_sq in action_sequences:
        uuid=get_uuid(ac_sq)
        r=client.get_or_create(endpoint='action',
            data={'action_sequence': uuid})
        for action in r:
            actions.append(action)
        
    action_units=[]  
    for action in actions:
        action_units.append(client.get_or_create(endpoint='action-unit',
            data={ 'action': action['uuid']}))
    
    #get action def description for each action
    action_defs=[]
    for i in actions:
        uuid=get_uuid(i['action_def'])
        print(uuid)
        r=client.get_or_create(endpoint='action-def', 
                         data={'uuid': uuid})
        action_defs.append(r[0]['description'])
    

    #format actions and parameters as a dictionary to export as json    
    json_output=[]

    for num, i in enumerate(action_units):
        if len(i)>1: #well-level actions
            output_entry=({'op': action_defs[num],#i[0]['description'].split(' ')[0],
                  'reagent': i[0]['source_material'],
                  'object': i[0]['destination_material'],
                   'wells': []})
            for j in i:
                try:
                    well=(j['description'].split(' ')[-1])
                    for k in range(len(j['parameter'])):
                        param_uuid=get_uuid(j['parameter'][k]['parameter_def'])
                        param=client.get_or_create(endpoint='parameter-def',
                                     data={'uuid': param_uuid})[0]
                        val=j['parameter'][k]['parameter_val_nominal']['value']
                        unit=j['parameter'][k]['parameter_val_nominal']['unit']     
                        quantity=format_units(val, unit)
                        output_entry['wells'].append({'well': well, param['description']: quantity})
                except AttributeError:
                    pass
            json_output.append(output_entry)
        else: #plate-level actions
            output_entry=({'op': action_defs[num],#i[0]['description'].split(' ')[0],
                  'reagent': i[0]['source_material'],
                  'object': i[0]['destination_material'],})
            for k in range(len(i[0]['parameter'])): #loop over all parameters
                param_uuid=get_uuid(i[0]['parameter'][k]['parameter_def'])
                param=client.get_or_create(endpoint='parameter-def',
                                   data={'uuid': param_uuid})[0]
                val=i[0]['parameter'][k]['parameter_val_nominal']['value']
                unit=i[0]['parameter'][k]['parameter_val_nominal']['unit']     
                quantity=format_units(val, unit)
                output_entry[param['description']]=quantity
            json_output.append(output_entry)
    return json_output

In [44]:
json_output = escalate_to_autoprotocol('test 2')

export_json(json_output, 'test_2_autoprotocol')

{'description': 'test 2'}
http://localhost:8000/api/experiment-instance/?description=test+2
GET: OK. Found 1 results
{'action_sequence': '62a8a44f-9e15-4030-9427-6b93ef02a50a'}
http://localhost:8000/api/action/?action_sequence=62a8a44f-9e15-4030-9427-6b93ef02a50a
GET: OK. Found 1 results
{'action_sequence': 'b2f8faf9-f804-47a4-970f-c24593069dec'}
http://localhost:8000/api/action/?action_sequence=b2f8faf9-f804-47a4-970f-c24593069dec
GET: OK. Found 1 results
{'action_sequence': 'a8247256-66fc-4d00-9ade-cc211c69f7d5'}
http://localhost:8000/api/action/?action_sequence=a8247256-66fc-4d00-9ade-cc211c69f7d5
GET: OK. Found 1 results
{'action_sequence': '95b4e618-803c-46ce-91b7-6c82910ba3fe'}
http://localhost:8000/api/action/?action_sequence=95b4e618-803c-46ce-91b7-6c82910ba3fe
GET: OK. Found 1 results
{'action_sequence': '22302949-a181-455a-af46-67e21a59a5b4'}
http://localhost:8000/api/action/?action_sequence=22302949-a181-455a-af46-67e21a59a5b4
GET: OK. Found 1 results
{'action_sequence': '18

[{'op': 'bring_to_temperature',
  'reagent': None,
  'object': 'http://localhost:8000/api/base-bom-material/c053aea3-23a1-4548-9370-e7f9b4a3c92a/',
  'temperature': '0.0:celcius'},
 {'op': 'stir',
  'reagent': None,
  'object': 'http://localhost:8000/api/base-bom-material/5d27bfbb-fc7b-4b0e-b34e-5af590e367ca/',
  'duration': '0.0:minute',
  'speed': '0.0:revolutions_per_minute'},
 {'op': 'stir',
  'reagent': None,
  'object': 'http://localhost:8000/api/base-bom-material/64ff44f9-76c0-4480-a05c-0e00e5277373/',
  'duration': '0.0:minute',
  'speed': '0.0:revolutions_per_minute'},
 {'op': 'bring_to_temperature',
  'reagent': None,
  'object': 'http://localhost:8000/api/base-bom-material/d0fe9ac8-7e98-4a3e-b446-b2306157af77/',
  'temperature': '0.0:celcius'},
 {'op': 'stir',
  'reagent': None,
  'object': 'http://localhost:8000/api/base-bom-material/f445022c-1e98-4b69-998c-8b20d2d30914/',
  'duration': '0.0:minute',
  'speed': '0.0:revolutions_per_minute'},
 {'op': 'stir',
  'reagent': Non